# Wrapping dependent libraries

**StructureAnalysis** is a set of libraries that include statistical models for the analysis of structured data (mainly sequences and tree-structured data):

* **StatTool** is a library containing classes concerning parametric  modeling of univariate and multivariate data.

* **SequenceAnalysis** is a library containing statistical functions and classes for markovian models (e.g., hidden variable-order Markov and hidden semi-Markov models) and multiple change-point models for sequences.
  The **SequenceAnalysis** library depends on the **StatTool** library.

These libraries have been extensively used for the identification and characterization of developmental patterns in plants from the tissular to the whole plant scale.
Previously interfaced with *AML* (a home-made, domain-specific programming language), some work has been done to switch to *Python*.
Nevertheless, the complexity of writing wrappers with **Boost.Python** limited the number of available components in *Python* in comparison to *AML*.
One advantage of having a statistical library written in *C++* available in *Python* is that developers can benefit from all *Python* packages.
As illustrated with the following figures this is particularly useful for providing visualizations for model quality assessment using -- for example -- the **Matplotlib** *Python* package.

In [1]:
import matplotlib
%matplotlib nbagg

/Users/pfernique/.miniconda/envs/FP17/lib/python2.7/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


We here aim at presenting how dependent libraries can be wrapped.
First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [2]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [3]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [4]:
import autowig

## The **StatTool** library

To compile and install the **StatTool** library, we use the available **Conda** recipes.

In [5]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\StructureAnalysis\stat_tool\bin\conda\libstat_tool -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../git/StructureAnalysis/stat_tool/bin/conda/libstat_tool -c statiskit -c conda-forge
!conda install -y libstat_tool --use-local -c statiskit -c conda-forge




####################################################################################
Source and build intermediates have been left in /Users/pfernique/.miniconda/conda-bld.
There are currently 19 accumulated.
To remove them, you can run the ```conda build purge``` command
Fetching package metadata .................

PackageNotFoundError: Packages missing in current channels:
            
  - libstat_tool

We have searched for the packages in the following channels:
            
  - file:///Users/pfernique/.miniconda/conda-bld/osx-64
  - file:///Users/pfernique/.miniconda/conda-bld/noarch
  - https://conda.anaconda.org/statiskit/osx-64
  - https://conda.anaconda.org/statiskit/noarch
  - https://conda.anaconda.org/conda-forge/osx-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.continuum.io/pkgs/main/osx-64
  - https://repo.continuum.io/pkgs/main/noarch
  - https://repo.continuum.io/pkgs/free/osx-64
  - https://repo.continuum.io/pkgs/free/noarch
  - https://repo.c

Then, we can proceed to the actual generation of wrappers for the **StatTool** library.
For this, we create an empty Abstract Semantic Graph (ASG).

In [6]:
import autowig
asg = autowig.AbstractSemanticGraph()

Then, we parse headers with relevant compilation flags.

In [7]:
%%time

import sys
try:
    from path import Path
except:
    from path import path as Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = list((prefix/'include'/'stat_tool').walkfiles('*.h*'))
else:
    headers = list((prefix/'include'/'stat_tool').walkfiles('*.h*'))
    
flags = ['-x', 'c++', '-std=c++11']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
autowig.parser.plugin = 'clanglite'
asg = autowig.parser(asg, headers,
                          flags = flags,
                          bootstrap = 2,
                          silent = True)

OSError: [Errno 2] No such file or directory: '/Users/pfernique/.miniconda/envs/FP17/include/stat_tool'

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation could be suitable.
Nevertheless, some **AutoWIG** limitations (**AutoWIG** doesn't have a complete knowledge concerning copyable classes) and the requirement of classes defined in the standard *C++* library lead us to implement a new `controller`.

In [ ]:
def stat_tool_controller(asg):
    for noncopyable in ['class ::std::basic_streambuf< char, struct ::std::char_traits< char > >',
                        'class ::std::codecvt< char, char, __mbstate_t >',
                        'class ::std::basic_filebuf< char, struct ::std::char_traits< char > >',
                        'class ::std::locale::facet',
                        'class ::std::locale::id',
                        'class ::std::ctype< char >',
                        'class ::std::ios_base',
                        'class ::std::basic_istream< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_ifstream< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_ostream< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_ostringstream< char, struct ::std::char_traits< char >, class ::std::allocator< char > >',
                        'class ::std::basic_ios< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_stringbuf< char, struct ::std::char_traits< char >, class ::std::allocator< char > >']:
        asg[noncopyable].is_copyable = False
    for cls in asg.classes():
        for fld in cls.fields(access='public'):
            if fld.qualified_type.unqualified_type.globalname == 'class ::std::locale::id':
                fld.boost_python_export = False
    for specialization in asg['class ::std::reverse_iterator'].specializations():
        specialization.boost_python_export = False
    asg['::std::ios_base::openmode'].qualified_type.boost_python_export = True
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [ ]:
%%time

autowig.controller['stat_tool'] = stat_tool_controller
autowig.controller.plugin = 'stat_tool'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
%%time

autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = os.path.join("..", "git", "StructureAnalysis", "stat_tool", "src", "py", "wrapper", "_stat_tool.cpp"),
                             decorator = os.path.join("..", "git", "StructureAnalysis", "stat_tool", "src", "py", "stat_tool", "_stat_tool.py"),
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
%%time

wrappers.write()

Here is the list of the generated wrappers (untracked files).

In [ ]:
if is_windows:
    !cd ..\git\StructureAnalysis & git status
else:
    !cd ../git/StructureAnalysis && git status

Here is an example of the generated wrappers.

In [ ]:
if is_windows:
    !pygmentize ..\git\StructureAnalysis\stat_tool\src\py\wrapper\_stat_tool.cpp
else:
    !pygmentize ../git/StructureAnalysis/stat_tool/src/py/wrapper/_stat_tool.cpp

In order to wrap a *C++* library, that will be used as a dependency by other libraries, the user needs to save the ASG resulting from the wrapping process.
We therefore use the **pickle** *Python* package for serializing the **StatTool** ASG in the `'StructureAnalysis/stat_tool/ASG.pkl'` file.

In [ ]:
import pickle
with open(os.path.join("..", "git", "StructureAnalysis", "stat_tool", "ASG.pkl"), "wb") as filehandler:
    pickle.dump(asg, filehandler)

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [ ]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\StructureAnalysis\stat_tool\bin\conda\python-stat_tool -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../git/StructureAnalysis/stat_tool/bin/conda/python-stat_tool -c statiskit -c conda-forge
!conda install -y python-stat_tool --use-local -c statiskit -c conda-forge

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
import stat_tool
import os
%reload_ext stat_tool.mplotlib
%reload_ext stat_tool.aml

In [ ]:
meri1 = stat_tool.Histogram(os.path.join("..", "git", "StructureAnalysis", "stat_tool", "share", "data", "meri1.his"))
meri2 = stat_tool.Histogram(os.path.join("..", "git", "StructureAnalysis", "stat_tool", "share", "data", "meri2.his"))
meri3 = stat_tool.Histogram(os.path.join("..", "git", "StructureAnalysis", "stat_tool", "share", "data", "meri3.his"))
meri4 = stat_tool.Histogram(os.path.join("..", "git", "StructureAnalysis", "stat_tool", "share", "data", "meri4.his"))
meri5 = stat_tool.Histogram(os.path.join("..", "git", "StructureAnalysis", "stat_tool", "share", "data", "meri5.his"))
meri0 = stat_tool.Merge(meri1, meri2, meri3, meri4, meri5)

In [ ]:
mixt = stat_tool.MixtureEstimation(meri0, 4, "BINOMIAL",
                                   display=True)
fig = mixt.plot()

## The **SequenceAnalysis** library

Once the wrapping of the **StatTool** library is performed, we need to compile and install the **SequenceAnalysis** library.
For this, we use the available **Conda** recipes.

In [ ]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\StructureAnalysis\sequence_analysis\bin\conda\libsequence_analysis
else:
    !conda build --python=$PYTHON_VERSION ../git/StructureAnalysis/sequence_analysis/bin/conda/libsequence_analysis
!conda install -y libsequence_analysis --use-local -c statiskit -c conda-forge

Then, we can proceed to the actual generation of wrappers for the **SequenceAnalysis** *C++* library.
In order to wrap a *C++* library that has dependencies, the user need to combine the ASGs resulting from the wrapping of its dependencies before performing its own wrapping.
For this, we create an empty Abstract Semantic Graph (ASG).

In [ ]:
asg = autowig.AbstractSemanticGraph()

Then, we use the **pickle** *Python* package for de-serializing the **StatTool** ASG and merge it in the current ASG.

In [ ]:
%%time

with open(os.path.join("..", "git", "StructureAnalysis", "stat_tool", "ASG.pkl"), "rb") as filehandler:
    asg.merge(pickle.load(filehandler))

Then, we parse headers with relevant compilation flags.

In [ ]:
%%time

if is_windows:
    headers = list((prefix/'include'/'sequence_analysis').walkfiles('*.h*'))
else:
    headers = list((prefix/'include'/'sequence_analysis').walkfiles('*.h*'))
    
flags = ['-x', 'c++', '-std=c++11']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
asg = autowig.parser(asg, headers,
                          flags = flags,
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [ ]:
%%time

autowig.controller.plugin = 'default'
asg = autowig.controller(asg, clean = False)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
%%time

autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = os.path.join("..", "git", "StructureAnalysis", "sequence_analysis", "src", "py", "wrapper", "_sequence_analysis.cpp"),
                             decorator = os.path.join("..", "git", "StructureAnalysis", "sequence_analysis", "src", "py", "sequence_analysis", "_sequence_analysis.py"),
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
%%time

wrappers.write()

Here is the list of the generated wrappers (untracked files).

In [ ]:
if is_windows:
    !cd ..\git\StructureAnalysis & git status
else:
    !cd ../git/StructureAnalysis && git status

Here is an example of the generated wrappers.

In [ ]:
if is_windows:
    !pygmentize ..\git\StructureAnalysis/sequence_analysis/src/py/wrapper/_sequence_analysis.cpp
else:
    !pygmentize ../git/StructureAnalysis/sequence_analysis/src/py/wrapper/_sequence_analysis.cpp

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [ ]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\StructureAnalysis\sequence_analysis\bin\conda\python-sequence_analysis -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../git/StructureAnalysis/sequence_analysis/bin/conda/python-sequence_analysis -c statiskit -c conda-forge
!conda install -y python-sequence_analysis --use-local -c statiskit -c conda-forge --force

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
import sequence_analysis
%reload_ext sequence_analysis.mplotlib
%reload_ext sequence_analysis.aml

In [ ]:
seq = sequence_analysis.Sequences(os.path.join("..", "git", "StructureAnalysis", "sequence_analysis", "share", "data", "well_log_filtered_indexed.seq"))

In [ ]:
seq.segmentation(1, 80, "GAUSSIAN", "LIKELIHOOD_SLOPE", min_nb_segment=30)